**WiFi Trilateration**

**Problem Statement**

We cannot imagine the world today without Global Navigation Satellite Systems (GNSS) systems like GPS, GLONASS, Galileo, NavIC. Except! when the view of the sky is obstructed

Indoor localisation is essential in our built-up environment, yet this technology has not caught up. Many attempts using ISM radio signals to localise indoors have had only moderate success. You may have heard of iBeacon or seen Google Assistant giving you suspiciously specific location notifications.

In this activity, you will attempt to localise within a controlled area using RSSI signals from nearby Access Points (AP) without the precise location of either the AP or the mobile phone.

Attached Files
[WiFiLocation.package.tar.xz](https://api.t.cyberthon24.ctf.sg/file?id=clu5qrt5m0eji080668p1zh9x&name=WiFiLocation.package.tar.xz)

**Solution**

We can do some simple regression to solve this challange

You need to put WiFiLocation.tar.xz into a folder <br>
Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!tar -xf /content/drive/MyDrive/Cyberthon/OutofPerspective.tar.xz

Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Read from the wifi_train.json file

In [ ]:
df0 = pd.read_json("package/wifi_train.json")

Create a list of all the access points names as well as their signals <br>
If an access point has multiple signals, we take the average 

In [ ]:
ap_list = []
ap_names = set()
for i in df0:
  ap = {'x': df0[i]['location']['x'], 'y': df0[i]['location']['y']}
  for j in df0[i]['data'].keys():
    ap_names.add(j)
    ap[j] = np.mean(df0[i]['data'][j])
  ap_list.append(ap)

We create the X train and Y train lists.

X train will consist of a list of every access points possible in the entire training dataset, even if they do not show up in that iteration

Y train will consist of the corresponding (x, y) of the mobile device

In [ ]:
stuff = {elem: 0 for elem in ap_names}
shuce = []
Y = []
X = []
for i in ap_list:
  temp = stuff.copy()
  for j, k in i.items():
    if not (j=='x'or j=='y'):
      temp[str(j)] = k
  Y.append([i['x'], i['y']])
  shuce.append(temp)
for i in shuce:
  X.append(list(i.values()))

Train the Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

x = np.array(X)
y = np.array(Y)

model = RandomForestRegressor()

model.fit(x, y)

Read from wifi_test.json

In [ ]:
df1 = pd.read_json("package/wifi_test.json")
ap_list2 = []

for i in df1:
  ap = {}
  for j in df1[i]['data'].keys():
    if j in ap_names:
      ap[j] = np.mean(df1[i]['data'][j])
  ap_list2.append(ap)

Prepocess wifi_test.csv to put into the model

In [ ]:
stuff2 = {elem: 0 for elem in ap_names}
shuce2 = []
X2 = []
for i in ap_list2:
  temp = stuff2.copy()
  for j, k in i.items():
    if not (j=='x'or j=='y'):
      temp[str(j)] = k
  shuce2.append(temp)
for i in shuce2:
  X2.append(list(i.values()))
print(X2[0])

Plug all the values from test.csv into the model

In [ ]:
outputX = []
outputY = []
for i in X2:
  out = model.predict([i])[0]
  outputX.append(out[0])
  outputY.append(out[1])

Lastly, write outputs into submission.csv

In [ ]:
df2 = pd.read_csv('package/submission.csv')
for i, row in df2.iterrows():
    df2['locationX'] = outputX
    df2['locationY'] = outputY

df2.to_csv('package/submission.csv', index=False)